In [7]:
from PIL import Image, ImageDraw
import os
from skimage import measure, color, io

In [8]:
# base, for new images resize
IMAGE_WIDTH = 1000
IMAGE_HEIGHT = 1000
IMAGE_HEIGHT_DOUBLE = IMAGE_HEIGHT * 2
IMAGE_WIDTH_DOUBLE = IMAGE_WIDTH * 2
# colors
BACKGROUND_COLOR = (255, 255, 255)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

# lambdas
one_eight_grid = lambda length: [(i * length // 8) for i in range(8)]
one_sixteenth_grid = lambda length: [(i * length // 16) for i in range(16)]
create_empty_image = lambda width, height, color: Image.new("RGB", (width, height), color)

### Ground Truth for the following set 

**Test10-17_Arda_synthetic_data/segmentations-original:**

- _s001 - 6 objects: 4 squares (10x10, black), 2 squares (20x20, black)
- _s002 - 5 objects: 1 square (50x50, black), 1 square (75x75, black), 1 square (60x60, black), 1 square (40x40, black), 1 square (80x80, black)
- _s003 - 1 object: 1 square (500x1000, black)
- _s004 - 0 objects (full black background)
- _s005 - 0 objects (empty white image)
- _s006 - 10 objects: 2 squares (20x20, black), 6 squares (10x10, black), 2 squares (20x20, black)
- _s007 - 10 objects: 2 squares (30x30, black), 6 squares (15x15, black), 2 squares (30x30, black)
- _s008 - 10 objects: 2 squares (40x40, black), 6 squares (20x20, black), 2 squares (40x40, black)

**Test10-17_Arda_synthetic_data/segmentations-doublesize:**

- _s001 - 6 objects: 4 squares (10x10, black), 2 squares (20x20, black)
- _s002 - 5 objects: 1 square (50x50, black), 1 square (75x75, black), 1 square (60x60, black), 1 square (40x40, black), 1 square (80x80, black)
- _s003 - 1 object: 1 square (500x1000, black)
- _s004 - 0 objects (full black background)
- _s005 - 0 objects (empty white image)
- _s006 - 10 objects: 2 squares (20x20, black), 6 squares (10x10, black), 2 squares (20x20, black)
- _s007 - 10 objects: 2 squares (30x30, black), 6 squares (15x15, black), 2 squares (30x30, black)
- _s008 - 10 objects: 2 squares (40x40, black), 6 squares (20x20, black), 2 squares (40x40, black)

In [9]:
_8 = one_eight_grid(IMAGE_WIDTH)
_16 = one_sixteenth_grid(IMAGE_WIDTH)
_16x2 = one_sixteenth_grid(IMAGE_WIDTH * 2)
_16x1_5 = one_sixteenth_grid(int(IMAGE_WIDTH * 1.5))

_16x4 = one_sixteenth_grid(IMAGE_WIDTH_DOUBLE * 2)
_16x3 = one_sixteenth_grid(int(IMAGE_WIDTH_DOUBLE * 1.5))


# All the datasets for testing.
# current set configuration is for 
# for 8 Images 
SET_CONFIGURATION = {
    "Test10-17_Arda_synthetic_data/segmentations-original": [
        {
            "image": {
                "width": IMAGE_WIDTH, 
                "height": IMAGE_HEIGHT, 
                "background_color": WHITE,
                "name": "Arda_s001" # 6 objects ~ black squares
            },
            "objects": [
                {
                    "type": "square",
                    "x": _16[7],
                    "y": _16[7],
                    "width": 10,
                    "height": 10,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[9],
                    "y": _16[7],
                    "width": 10,
                    "height": 10,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[7],
                    "y": _16[9],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[9],
                    "y": _16[9],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[7],
                    "y": _16[11],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[9],
                    "y": _16[11],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                }
            ]
        },
        {
            "image": {
                "width": IMAGE_WIDTH , 
                "height": IMAGE_HEIGHT , 
                "background_color": WHITE,
                "name": "Arda_s002" # stretch image
            },
            "objects": [
                {
                    "type": "square",
                    "x": 0,
                    "y": 10,
                    "width": 1000,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": 10,
                    "y": 0,
                    "width": 20,
                    "height": 1000,
                    "color": BLACK,
                }
                
            ]
        },
        {
            "image": {
                "width": IMAGE_WIDTH, 
                "height": IMAGE_HEIGHT, 
                "background_color": WHITE,
                "name": "Arda_s003" # half image 
            },
            "objects": [
                {
                    "type": "square",
                    "x": 0,
                    "y": 0,
                    "width": 500,
                    "height": 1000,
                    "color": BLACK,
                },
                
            ]

        },
        {
            "image": {
                "width": IMAGE_WIDTH, 
                "height": IMAGE_HEIGHT, 
                "background_color": BLACK,
                "name": "Arda_s004" # full image
            },
            "objects": [

                
            ]
        },
        {
            "image": {
                "width": IMAGE_WIDTH, 
                "height": IMAGE_HEIGHT, 
                "background_color": WHITE,
                "name": "Arda_s005" # empty image
            },
            "objects": [
            ]
        },
        {
            "image": {
                "width": IMAGE_WIDTH, 
                "height": IMAGE_HEIGHT, 
                "background_color": WHITE,
                "name": "Arda_s006" # ribbed pattern
            },
            "objects": [
                {
                    "type": "square",
                    "x": _16[7],
                    "y": _16[4],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[9],
                    "y": _16[4],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[7],
                    "y": _16[6],
                    "width": 10,
                    "height": 10,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[9],
                    "y": _16[6],
                    "width": 10,
                    "height": 10,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[7],
                    "y": _16[7],
                    "width": 10,
                    "height": 10,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[9],
                    "y": _16[7],
                    "width": 10,
                    "height": 10,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[7],
                    "y": _16[8],
                    "width": 10,
                    "height": 10,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[9],
                    "y": _16[8],
                    "width": 10,
                    "height": 10,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[7],
                    "y": _16[10],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16[9],
                    "y": _16[10],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                }
            ]
        },
        {
            "image": {
                "width": int(IMAGE_WIDTH * 1.5), 
                "height": int(IMAGE_HEIGHT * 1.5), 
                "background_color": WHITE,
                "name": "Arda_s007" # ribbed pattern * 1.5
            },
            "objects": [
                {
                    "type": "square",
                    "x": _16x1_5[7],
                    "y": _16x1_5[4],
                    "width": 30,
                    "height": 30,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x1_5[9],
                    "y": _16x1_5[4],
                    "width": 30,
                    "height": 30,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x1_5[7],
                    "y": _16x1_5[6],
                    "width": 15,
                    "height": 15,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x1_5[9],
                    "y": _16x1_5[6],
                    "width": 15,
                    "height": 15,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x1_5[7],
                    "y": _16x1_5[7],
                    "width": 15,
                    "height": 15,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x1_5[9],
                    "y": _16x1_5[7],
                    "width": 15,
                    "height": 15,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x1_5[7],
                    "y": _16x1_5[8],
                    "width": 15,
                    "height": 15,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x1_5[9],
                    "y": _16x1_5[8],
                    "width": 15,
                    "height": 15,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x1_5[7],
                    "y": _16x1_5[10],
                    "width": 30,
                    "height": 30,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x1_5[9],
                    "y": _16x1_5[10],
                    "width": 30,
                    "height": 30,
                    "color": BLACK,
                }
            ]
        },
        {
            "image": {
                "width": IMAGE_WIDTH * 2, 
                "height": IMAGE_HEIGHT * 2, 
                "background_color": WHITE,
                "name": "Arda_s008" # ribbed pattern * 2
            },
            "objects": [
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[4],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[4],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[6],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[6],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[7],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[7],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[8],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[8],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[10],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[10],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                }
            ]
        }
    ],
    "Test10-17_Arda_synthetic_data/segmentations-double": [
        {
            "image": {
                "width": IMAGE_WIDTH_DOUBLE, 
                "height": IMAGE_HEIGHT_DOUBLE, 
                "background_color": WHITE,
                "name": "Arda_s001" # 6 objects ~ black squares
            },
            "objects": [
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[7],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[7],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[9],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[9],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[11],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[11],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                }
            ]
        },
        {
            "image": {
                "width": IMAGE_WIDTH_DOUBLE , 
                "height": IMAGE_HEIGHT_DOUBLE , 
                "background_color": WHITE,
                "name": "Arda_s002" # stretch image
            },
            "objects": [
                {
                    "type": "square",
                    "x": 0,
                    "y": 10,
                    "width": 2000,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": 10,
                    "y": 0,
                    "width": 20,
                    "height": 2000,
                    "color": BLACK,
                }
                
            ]
        },
        {
            "image": {
                "width": IMAGE_WIDTH_DOUBLE, 
                "height": IMAGE_HEIGHT_DOUBLE, 
                "background_color": WHITE,
                "name": "Arda_s003" # half image 
            },
            "objects": [
                {
                    "type": "square",
                    "x": 0,
                    "y": 0,
                    "width": 1000,
                    "height": 2000,
                    "color": BLACK,
                },
                
            ]

        },
        {
            "image": {
                "width": IMAGE_WIDTH_DOUBLE, 
                "height": IMAGE_HEIGHT_DOUBLE, 
                "background_color": BLACK,
                "name": "Arda_s004" # full image
            },
            "objects": [

                
            ]
        },
        {
            "image": {
                "width": IMAGE_WIDTH_DOUBLE, 
                "height": IMAGE_HEIGHT_DOUBLE, 
                "background_color": WHITE,
                "name": "Arda_s005" # empty image
            },
            "objects": [
            ]
        },
        {
            "image": {
                "width": IMAGE_WIDTH_DOUBLE, 
                "height": IMAGE_HEIGHT_DOUBLE, 
                "background_color": WHITE,
                "name": "Arda_s006" # ribbed pattern
            },
            "objects": [
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[4],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[4],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[6],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[6],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[7],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[7],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[8],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[8],
                    "width": 20,
                    "height": 20,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[7],
                    "y": _16x2[10],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x2[9],
                    "y": _16x2[10],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                }
            ]
        },
        {
            "image": {
                "width": int(IMAGE_WIDTH_DOUBLE * 1.5), 
                "height": int(IMAGE_HEIGHT_DOUBLE * 1.5), 
                "background_color": WHITE,
                "name": "Arda_s007" # ribbed pattern * 1.5
            },
            "objects": [
                {
                    "type": "square",
                    "x": _16x3[7],
                    "y": _16x3[4],
                    "width": 60,
                    "height": 60,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x3[9],
                    "y": _16x3[4],
                    "width": 60,
                    "height": 60,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x3[7],
                    "y": _16x3[6],
                    "width": 30,
                    "height": 30,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x3[9],
                    "y": _16x3[6],
                    "width": 30,
                    "height": 30,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x3[7],
                    "y": _16x3[7],
                    "width": 30,
                    "height": 30,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x3[9],
                    "y": _16x3[7],
                    "width": 30,
                    "height": 30,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x3[7],
                    "y": _16x3[8],
                    "width": 30,
                    "height": 30,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x3[9],
                    "y": _16x3[8],
                    "width": 30,
                    "height": 30,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x3[7],
                    "y": _16x3[10],
                    "width": 60,
                    "height": 60,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x3[9],
                    "y": _16x3[10],
                    "width": 60,
                    "height": 60,
                    "color": BLACK,
                }
            ]
        },
        {
            "image": {
                "width": IMAGE_WIDTH_DOUBLE * 2, 
                "height": IMAGE_HEIGHT_DOUBLE * 2, 
                "background_color": WHITE,
                "name": "Arda_s008" # ribbed pattern * 2
            },
            "objects": [
                {
                    "type": "square",
                    "x": _16x4[7],
                    "y": _16x4[4],
                    "width": 80,
                    "height": 80,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x4[9],
                    "y": _16x4[4],
                    "width": 80,
                    "height": 80,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x4[7],
                    "y": _16x4[6],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x4[9],
                    "y": _16x4[6],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x4[7],
                    "y": _16x4[7],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x4[9],
                    "y": _16x4[7],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x4[7],
                    "y": _16x4[8],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x4[9],
                    "y": _16x4[8],
                    "width": 40,
                    "height": 40,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x4[7],
                    "y": _16x4[10],
                    "width": 80,
                    "height": 80,
                    "color": BLACK,
                },
                {
                    "type": "square",
                    "x": _16x4[9],
                    "y": _16x4[10],
                    "width": 80,
                    "height": 80,
                    "color": BLACK,
                }
            ]
        }
    ],

}

In [6]:
_16x2

[0,
 125,
 250,
 375,
 500,
 625,
 750,
 875,
 1000,
 1125,
 1250,
 1375,
 1500,
 1625,
 1750,
 1875]

In [87]:
one_eight_grid(IMAGE_WIDTH)

[0, 125, 250, 375, 500, 625, 750, 875]

In [94]:
_116 = one_sixteenth_grid(IMAGE_WIDTH)

_116[9]

562

In [10]:
to_verify_images = []

for dataset_name, images in SET_CONFIGURATION.items():
    dataset_dir = f"./tests/test_data/{dataset_name}"
    os.makedirs(dataset_dir, exist_ok=True)
    for idx, img_data in enumerate(images):
        
        img_config = img_data["image"]
        objects = img_data["objects"]
        
        img = Image.new("RGB", (img_config["width"], img_config["height"]), img_config["background_color"])
        draw = ImageDraw.Draw(img)
        
        # drawing the objects as rectangles 
        for obj in objects:
            if obj["type"] == "square":
                draw.rectangle([obj["x"], obj["y"], obj["x"] + obj["width"] - 1, obj["y"] + obj["height"] - 1], fill=obj["color"])
        
        img_path = f"{dataset_dir}/{img_config['name']}.png"
        img.save(img_path)
        to_verify_images.append(img_path)
        print(f"saved {img_path}")

saved ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-original/Arda_s001.png
saved ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-original/Arda_s002.png
saved ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-original/Arda_s003.png
saved ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-original/Arda_s004.png
saved ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-original/Arda_s005.png
saved ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-original/Arda_s006.png
saved ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-original/Arda_s007.png
saved ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-original/Arda_s008.png
saved ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-double/Arda_s001.png
saved ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-double/Arda_s002.png
saved ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-double/Arda_s003.png
saved

In [ ]:
for img_path in to_verify_images:
    print(f"\nAnalyzing: {img_path}")

    img = io.imread(img_path)
    if len(img.shape) == 3:
        img_gray = color.rgb2gray(img)
    else:
        img_gray = img
    
    # Threshold to get binary image (assuming black objects on white background)
    binary = img_gray < 0.5  # Black pixels are True
    
    labeled_img = measure.label(binary)
    
    # Get region properties
    regions = measure.regionprops(labeled_img)
    
    print(f"Found {len(regions)} objects:")
    for i, region in enumerate(regions):
        print(f"  Object {i+1}: area={region.area}, bbox={region.bbox}, centroid={region.centroid}")


Analyzing: ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-original/Arda_s001.png
Found 6 objects:
  Object 1: area=100.0, bbox=(437, 437, 447, 447), centroid=(np.float64(441.5), np.float64(441.5))
  Object 2: area=100.0, bbox=(437, 562, 447, 572), centroid=(np.float64(441.5), np.float64(566.5))
  Object 3: area=400.0, bbox=(562, 437, 582, 457), centroid=(np.float64(571.5), np.float64(446.5))
  Object 4: area=400.0, bbox=(562, 562, 582, 582), centroid=(np.float64(571.5), np.float64(571.5))
  Object 5: area=400.0, bbox=(687, 437, 707, 457), centroid=(np.float64(696.5), np.float64(446.5))
  Object 6: area=400.0, bbox=(687, 562, 707, 582), centroid=(np.float64(696.5), np.float64(571.5))

Analyzing: ./tests/test_data/Test10-17_Arda_synthetic_data/segmentations-original/Arda_s002.png
Found 1 objects:
  Object 1: area=39600.0, bbox=(0, 0, 1000, 1000), centroid=(np.float64(261.92424242424244), np.float64(261.92424242424244))

Analyzing: ./tests/test_data/Test10-17_Arda_syntheti